# Section 0

QPM: Assignment 7

DENG Yunyun <br>
LI Jiaxin <br>
SBAI Ilyas <br>
ZHOU Zhichen <br>

Note: NA

# Section 0.bis

We retrieve the excel data for the 9 factors and store it in the dataframe ```ret```

In [1]:
import pandas as pd
import numpy as np
from scipy.optimize import minimize

In [2]:
ret = pd.read_excel('QPM-FactorsData.xlsx')
ret = ret.set_index('Dates')
ret.describe().round(3)

,Market,SMB,HML,RMW,CMA,UMD,ROE,IA,BAB
count,647.000,647.000,647.000,647.000,647.000,647.000,647.000,647.000,647.000
mean,0.006,0.002,0.002,0.003,0.003,0.006,0.005,0.003,0.009
std,0.046,0.031,0.029,0.022,0.020,0.043,0.026,0.019,0.034
min,-0.232,-0.169,-0.140,-0.183,-0.069,-0.344,-0.145,-0.072,-0.157
25%,-0.021,-0.017,-0.014,-0.009,-0.010,-0.010,-0.007,-0.008,-0.005
50%,0.009,0.001,0.002,0.002,0.001,0.007,0.006,0.003,0.010
75%,0.036,0.020,0.017,0.013,0.015,0.029,0.019,0.015,0.025
max,0.161,0.217,0.129,0.133,0.096,0.184,0.104,0.092,0.154


# Section 1

Dividend discount model can explain why these factors are related to stock returns: <br>
>$\frac{M_t}{B_t}=\frac{\sum^{\infty}_{\tau = 1} E(Y_{t+\tau} - dB_{t+\tau})/(1+r)^{\tau}}{B_t}$ (1)

$M_t$ is the share price, $B_t$ is the total book equity, $r$ is the long-term average expected stock return, $Y_{t+\tau}$ is the total equity earnings for period $t+\tau$ and $dB_{t+\tau}$ is the change in total book equity

1. **Market (Market Risk)**<br>
    Investors demand a risk premium for taking market risk.
2. **SMB (Small Minus Big - Market Cap)**<br>
    Small-cap stocks have historically outperformed large-cap stocks, possibly due to their higher risk and growth potential.
3. **HML (High Minus Low -  book-to-market ratio)**<br> In equation (1), fix everything, a lower value of $M_t$, equivalently a higher book-to-market ratio implies a higher expected return
4. **RMW (Robust Minus Weak - Profitability)**<br> In equation (1), fix everything, a higher value of $E(Y_{t+\tau})$, equivalently a higher expected future earnings - profitiability - implies a higher expected
5. **CMA (Conservatice Minus Aggressive - Investment)** <br>
    In equation (1), fix everything, a higher value of $E(dB_{t+\tau})$, equivalently a higher expected growth in book equity - investment - implies a higher expected 
6. **UMD (Up minus Down - Momentum)**<br>
    Assets that have performed well in the recent past tend to continue performing well (or vice versa)
7. **ROE (Return on Equity - Profitability)** <br>
    As another factor for profitability, ROE shares the same rationale as that of RMW.  The high discount rates are necessary to offset the high expected ROE to induce low net present values of new capital and low investment.
8. **IA (Investment to Assets)**<br>
    Similar to CMA, the IA factor predicts returns because given expected cash flows, high costs of capital (r) implies low net present values of new capital and low investment.
9. **BAB (Bet Against Beta)** <br>
    Low-beta stocks have historically provided similar or better risk-adjusted returns compared to high-beta stocks





# Section 2

The resultant standardized characteristic is a long-short portfolio and all 9 factors are the return of a long-short portfolio -> the 9 characteristics are standardized already

At time t, each factor value in the dataset is the characteristic return of 2000 stocks on a long-short portfolio that longs stocks whose characteristic is above the cross-sectional average and shorts stocks whose characteristic is below cross-sectional average, or is equivalently $F_{i,t}r_{i,t-1}$, where $r_{i,t-1}$ is the return on stocks, $F_{i,t}$ is the long-short characteristic portfolio

In [3]:
#assign parameters
#number of factors
k = len(ret.columns)-1

#number of stock
N = 2000

#risk aversion
gamma = 5

We then compute the parametric-portfolio return and its mean and variance for the optimization problem of a mean-variance investor:
>$r_{p,t+1}(\theta) = r_{b,t+1} + \frac{1}{N_t}\theta ^T F_t^T r_{t+1}$ (2)

In [4]:
#assume that the investor's benchmark is the market portfolio
#define the objective mean-variance utility function for the remaining 8 factors
def mv_utility(theta, gamma):
    ret_ben = ret.iloc[:,0].mean()
    ret_cha = ret.iloc[:,1:].mean()
    return_para = ret.iloc[:,0] + (ret.iloc[:,1:]@theta/N)
    portfolio_return = ret_ben + (ret_cha@theta/N)
    portfolio_risk = np.var(return_para)
    return (gamma/2) * portfolio_risk - portfolio_return

In [5]:
#initial guess of theta
initial_theta = [1.5]*k
optimal_theta = minimize(mv_utility, initial_theta, args=(gamma,), method='Nelder-Mead')
optimal_theta.x

array([-1305.93495225,  -584.90878059, -2500.63153553,  2083.82990274,
        -349.01966673,  1974.42044173,  -577.78919476,  2082.24573448])

In [6]:
#rescale theta
rescaled_theta = optimal_theta.x/2000
rescaled_theta

array([-0.65296748, -0.29245439, -1.25031577,  1.04191495, -0.17450983,
        0.98721022, -0.2888946 ,  1.04112287])

# Section 3

Compute the Sharpe ratio

In [7]:
SR_fac = ret.mean()/ret.std()
SR_fac

Market    0.124969
SMB       0.057118
HML       0.080226
RMW       0.121442
CMA       0.137945
UMD       0.146440
ROE       0.197542
IA        0.180832
BAB       0.259683
dtype: float64

In [8]:
return_para = ret.iloc[:,0] + (ret.iloc[:,1:]@(optimal_theta.x)/N)
SR_para = np.mean(return_para)/np.std(return_para)
SR_para

0.27442438973004935

The sharpe ratio of the parametric portfolio is larger than the sharpe ratio for each of the nine factors.

# Section 4

Rewrite equation (2) assuming that the market portfolio is the benchmark: 
>$r_{p,t+1} = r_{m,t+1} + \sum^{N_t}_{i=1} (\frac{1}{N_t}\theta ^T F_{i,t}) r_{i,t+1}$ (3)

where $ \frac{1}{N_t}\theta ^T F_t^T$ is the weight vector for the 2000 assets in our optimal mean-variance portfolio.

Consequently, we can obtain the optimal portfolio weights for the assets by multiplying the optimal $\theta$ vector with characteristics vector $F_t$ scaled by the numer of assets $N_t$